# Catalyst Cooperative Jupyter Notebook Template
This notebook lays out a standard format and some best practices for creating interactive / exploratory notebooks which can be relatively easily shared between different PUDL users, and turned into reusable Python modules for integration into our underlying Python packages.

## Begin with a narrative outline
Each notebook should start with a brief explanation (in Markdown) explaining the purpose of the analysis, and outlining the different stages / steps which are taken to accomplish the analysis.
As the analysis develops, you can add new sections or details to this section.

## Notebooks should be runnable
Insofar as it's possible, another PUDL user who has cloned the repository that the notebook is part of should be able to update their `pudl-dev` conda environment, open the notebook, and run all cells successfully.
If there are required data or other prerequisites that the notebook cannot manage on its own -- like a file that needs to be downloaded by hand and placed in a particular location -- those steps should be laid out clearly at the beginning of the notebook.

## Avoid Troublesome Elements

### Don't hardcode passwords or access tokens
Most of our work is done in public Github repositories.
No authentication information should ever appear in a notebook.
These values can be stored in environment variables on your local computer.

### Do not hardocde values, especially late in the notebook
If the analysis depends on particular choices of input values, those should be called out explicitly at the beginning of the notebook.
(NB: We should explore ways to parameterize notebooks, [papermill](https://papermill.readthedocs.io/en/latest/) is one tool that does this).

### Don't hardcode absolute file paths
If anyone is going to be able to use the notebook, the files it uses will need to be stored somewhere that makes sense on both your and other computers.
We assume that anyone using this template has the PUDL package installed, and has a local PUDL data management environment set up.
  * Input data that needs to be stored on disk and accessed via a shared location should be saved under `<PUDL_IN>/data/local/<data_source>/`.
  * Intermediate saved data products (e.g. a pickled result of a computationally intensive process) and results should be saved to a location relative to the notebook, and within the directory hierarchy of the repository that the notebook is part of.
  
### Don't require avoidable long-running computations
Consider persisting to disk the results of computations that take more than a few minutes, if the outputs are small enough to be checked into the repository and shared with other users.
Only require the expensive computation to be run if this pre-computed output is not available.

### Don't litter
Don't leave lots of additional code laying around, even commented out, "just in case" you want to look at it again later.
Notebooks need to be relatively linear in the end, even though the thought processes and exploratory analyses that generate them may not be.
Once you have a working analysis, either prune those branches, or encapsulate them as options within functions.

### Don't load unneccesary libraries
Only import libraries which are required by the notebook, to avoid unnecessary dependencies.
If your analysis requires a new library that isn't yet part of the shared `pudl-dev` environment, add it to the `devtools/environment.yml` file so that others will get it when they update their environment.

## Related Resources:
Lots of these guidelines are taken directly from Emily Riederer's post: [RMarkdown Driven Development](https://emilyriederer.netlify.app/post/rmarkdown-driven-development/).
For more in depth explanation of the motivations behind this layout, do go check it out!

# Import Libraries
* Because it's very likely that you will be editing the PUDL Python packages or your own local module under development while working in the notebook, use the %autoreload magic with autoreload level 2 to ensure that any changes you've made in those files are always reflected in the code that's running in the notebook.
* Put all import statements at the top of the notebook, so everyone can see what its dependencies are up front, and so that if an import is going to fail, it fails early, before the rest of the notebook is run.
* Try to avoid importing individual functions and classes from deep within packages, as it may not be clear to other users where those elements came from, later in the notebook, and also to minimize the chance of namespace collisions.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Standard libraries
import logging
import os
import pathlib
import sys

# 3rd party libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl

# Configure Display Parameters

In [4]:
sns.set()
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,4)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Use Python Logging facilities
* Using a logger from the beginning will make the transition into the PUDL package easier.
* Creating a logging handler here will also allow you to see the logging output coming from PUDL and other underlying packages.

In [5]:
# logger=logging.getLogger()
# logger.setLevel(logging.DEBUG)
# handler = logging.StreamHandler(stream=sys.stdout)
# formatter = logging.Formatter('%(message)s')
# handler.setFormatter(formatter)
# logger.handlers = [handler]

# Define Functions
In many cases, the eventual product of a notebook analysis is going to be the creation of new, reusable functions that are integrated into the underlying PUDL code. You should begin the process of accumulating and organizing those functions as soon as you notice repeated patterns in your code.
* Functions should be used to encapsulate any potentially reusable code.
* Functions should be defined immediately after the imports, to avoid accidental dependence on zombie variables that are defined further down in the code
* While they may evolve over time, having brief docstrings explaining what they do will help others understand them.
* If there's a particular type of plot or visualization that is made repeatedly in the notebook, it's good to parameterize and functionalize it here too, so that as you refine the presentation of the data and results, those improvements can be made in a single place, and shown uniformly throughout the notebook.
* As these functions mature and become more general purpose tools, you will probably want to start migrating them into their own local module, under a `src` directory in the same directory as the notebook. You will want to import this module 

## Dummy EIA 861 ETL

In [6]:
def test_etl_eia(eia_inputs, pudl_settings):
    """
    This is a dummy function that runs the first part of the EIA ETL
    process -- everything up until the entity harvesting begins. For
    use in this notebook only.

    """
    eia860_tables = eia_inputs["eia860_tables"]
    eia860_years = eia_inputs["eia860_years"]
    eia861_tables = eia_inputs["eia861_tables"]
    eia861_years = eia_inputs["eia861_years"]
    eia923_tables = eia_inputs["eia923_tables"]
    eia923_years = eia_inputs["eia923_years"]

    # generate CSVs for the static EIA tables, return the list of tables
    #static_tables = _load_static_tables_eia(datapkg_dir)

    # Extract EIA forms 923, 860
    eia860_raw_dfs = pudl.extract.eia860.Extractor().extract(eia860_years, testing=True)
    eia861_raw_dfs = pudl.extract.eia861.Extractor().extract(eia861_years, testing=True)
    eia923_raw_dfs = pudl.extract.eia923.Extractor().extract(eia923_years, testing=True)

    # Transform EIA forms 860, 861, 923
    eia860_transformed_dfs = pudl.transform.eia860.transform(eia860_raw_dfs, eia860_tables=eia860_tables)
    eia861_transformed_dfs = pudl.transform.eia861.transform(eia861_raw_dfs, eia861_tables=eia861_tables)
    eia923_transformed_dfs = pudl.transform.eia923.transform(eia923_raw_dfs, eia923_tables=eia923_tables)

    # create an eia transformed dfs dictionary
    eia_transformed_dfs = eia860_transformed_dfs.copy()
    eia_transformed_dfs.update(eia861_transformed_dfs.copy())
    eia_transformed_dfs.update(eia923_transformed_dfs.copy())

    # convert types..
    eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(eia_transformed_dfs, 'eia')

    return eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs

# Define Notebook Parameters
If there are overarching parameters which determine the nature of the analysis -- which US states to look at, which utilities are of interest, a particular start and end date -- state those clearly at the beginning of the analysis, so that they can be referred to by the rest of the notebook and easily changed if need be.
* If the analysis depends on local (non-PUDL managed) datasets, define the paths to those data here.
* If there are external URLs or other resource locations that will be accessed, define those here as well.
* This is also where you should create your `pudl_settings` dictionary and connections to your local PUDL databases

In [7]:
EIA861_YEARS = list(range(2001, 2019))
pudl_settings = pudl.workspace.setup.get_defaults()
display(pudl_settings)

ferc1_engine = sa.create_engine(pudl_settings['ferc1_db'])
display(ferc1_engine)

pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
display(pudl_engine)

pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine)


# Is there other external data you need to pull in?
# If so, put it in a (relatively) standard place on the filesystem.
my_new_data_url = "https://mynewdata.website.gov/path/to/new/data.csv"
my_new_datadir = pathlib.Path(pudl_settings["data_dir"]) / "local/new_data_source"

# Store API keys and other secrets in environment variables
# and read them in here, if needed:
# API_KEY_EIA = os.environ["API_KEY_EIA "]
# API_KEY_FRED = os.environ["API_KEY_FRED "]

{'pudl_in': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR',
 'data_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/data',
 'settings_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/settings',
 'pudl_out': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR',
 'sqlite_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite',
 'parquet_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/parquet',
 'datapkg_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/datapkg',
 'notebook_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/notebook',
 'ferc1_db': 'sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/pudl.sqlite'}

Engine(sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/ferc1.sqlite)

Engine(sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/pudl.sqlite)

# Load Data
* Given the above parameters and functions, it should now be possible to load the required data into local variables for further wrangling, analysis, and visualization.
* If the data is not yet present on the machine of the person running the notebook, this step should also acquire the data from its original source, and place it in the appropriate location under `<PUDL_IN>/data/local/`.
* If there are steps which have to be done manually here, put them first so that they fail first if the user hasn't read the instructions, and they can fix the situation before a bunch of other work gets done. Try to minimize the amount of work in the filesystem that has to be done manually though.
* If this process takes a little while, don't be shy about producing `logging` output.
* Using the `%%time` cell magic can also help users understand which pieces of work / data acquisition are hard:

## EIA 861 (2010-2018)
* Not yet fully integrated into PUDL
* Post-transform harvesting process isn't compatible w/ EIA 861 structure
* Only getting the `sales_eia861`, `balancing_authority_eia861`, and `service_territory_eia861` tables

### Already Transformed EIA 861 DataFrames

In [8]:
%%time
eia_inputs = {
    "eia860_years": [],
    "eia860_tables": pudl.constants.pudl_tables["eia860"],
    "eia861_years": EIA861_YEARS,
    "eia861_tables": pudl.constants.pudl_tables["eia861"],
    "eia923_years": [],
    "eia923_tables": pudl.constants.pudl_tables["eia923"],
}
eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs = test_etl_eia(eia_inputs=eia_inputs, pudl_settings=pudl_settings)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
The following reported NERC regions are not currently recognized and become UNK values: ['CLECO']
CPU times: user 2min 24s, 

# Sanity Check Data
If there's any validation that can be done on the data which you've loaded to flag if/when it is inappropriate for the analysis that follows, do it here. If you find the data is unusable, use `assert` statements or `raise` Exceptions to stop the notebook from proceeding, and indicate what the problem is.

In [87]:
sorted(list(eia_transformed_dfs.keys()))

['advanced_metering_infrastructure_eia861',
 'balancing_authority_assn_eia861',
 'balancing_authority_eia861',
 'demand_response_eia861',
 'distribution_systems_eia861',
 'dynamic_pricing_eia861',
 'green_pricing_eia861',
 'mergers_eia861',
 'net_metering_eia861',
 'non_net_metering_eia861',
 'operational_data_eia861',
 'operational_data_other_eia861',
 'operational_data_revenue_eia861',
 'sales_eia861',
 'service_territory_eia861']

In [91]:
test_df = eia_transformed_dfs['demand_response_eia861'].copy()
raw_df = eia861_raw_dfs['demand_response_eia861'].copy()

In [94]:
idx_cols = [
    'utility_id_eia',
    'report_date',
    'state',
    'balancing_authority_code_eia'
]

In [95]:
raw_df = pudl.transform.eia861._early_transform(raw_df)
#raw_df = raw_df.dropna(subset=['utility_id_eia'])
#raw_df = raw_df[raw_df['utility_id_eia']!= 88888]
#raw_df = raw_df[raw_df['utility_id_eia']!= 99999]
raw_df = raw_df.fillna('temp')
#raw_df = raw_df.set_index(idx_cols)
raw_df = raw_df.drop_duplicates()
#raw_df = raw_df.reset_index()

In [96]:
#df = raw_df.set_index(idx_cols)
# raw_df['duped'] = raw_df.duplicated()
# raw_df[raw_df['duped']==True]

In [80]:
#raw_df= pudl.transform.eia861._clean_nerc(raw_df, idx_cols)

In [97]:
len(raw_df)

2653

In [98]:
len(raw_df.groupby(idx_cols))

2649

In [99]:
dd = raw_df.groupby(idx_cols).count()
#dd[dd.eq(2).any(1)]
dd[dd.iloc[:,1]>1]     

commercial_actual_peak_demand_savings_mw  \
utility_id_eia report_date state balancing_authority_code_eia                                             
40211          2013-01-01  IN    temp                                                                 2   
               2014-01-01  IN    temp                                                                 2   
               2015-01-01  IN    temp                                                                 2   
               2016-01-01  IN    temp                                                                 2   

                                                               commercial_customer_incentives_cost  \
utility_id_eia report_date state balancing_authority_code_eia                                        
40211          2013-01-01  IN    temp                                                            2   
               2014-01-01  IN    temp                                                            2   
               2015-01-01  IN    temp                                                            2   
               2016-01-01  IN    temp                                                            2   

                                                               commercial_customers  \
utility_id_eia report_date state balancing_authority_code_eia                         
40211          2013-01-01  IN    temp                                             2   
               2014-01-01  IN    temp                                             2   
               2015-01-01  IN    temp                                             2   
               2016-01-01  IN    temp                                             2   

                                                               commercial_energy_savings_mwh  \
utility_id_eia report_date state balancing_authority_code_eia                                  
40211          2013-01-01  IN    temp                                                      2   
               2014-01-01  IN    temp                                                      2   
               2015-01-01  IN    temp                                                      2   
               2016-01-01  IN    temp                                                      2   

                                                               commercial_other_costs  \
utility_id_eia report_date state balancing_authority_code_eia                           
40211          2013-01-01  IN    temp                                               2   
               2014-01-01  IN    temp                                               2   
               2015-01-01  IN    temp                                               2   
               2016-01-01  IN    temp                                               2   

                                                               commercial_potential_peak_demand_savings_mw  \
utility_id_eia report_date state balancing_authority_code_eia                                                
40211          2013-01-01  IN    temp                                                                    2   
               2014-01-01  IN    temp                                                                    2   
               2015-01-01  IN    temp                                                                    2   
               2016-01-01  IN    temp                                                                    2   

                                                               industrial_actual_peak_demand_savings_mw  \
utility_id_eia report_date state balancing_authority_code_eia                                             
40211          2013-01-01  IN    temp                                                                 2   
               2014-01-01  IN    temp                                                                 2   
               2015-01-01  IN    temp                                              

In [68]:
df = raw_df.loc[(raw_df['utility_id_eia']==306)] #.drop_duplicates(subset=idx_cols, keep='first')
df.set_index(idx_cols)

commercial_customers  \
utility_id_eia report_date state balancing_authority_code_eia business_model service_type                        
306.0          2003-01-01  TN    temp                         A              Bundled                      temp   
                                                                             Bundled                      temp   
                                                                             Bundled                      temp   
               2005-01-01  TN    temp                         A              Bundled                      temp   
                                                                             Bundled                      temp   
                                                                             Bundled                      temp   
                                                                             Bundled                      temp   
               2006-01-01  TN    temp                         A              Bundled                      temp   
                                                                             Bundled                      temp   
                                                                             Bundled                      temp   
                                                                             Bundled                      temp   
               2007-01-01  TN    temp                         A              Bundle                          0   

                                                                                          commercial_sales_mwh  \
utility_id_eia report_date state balancing_authority_code_eia business_model service_type                        
306.0          2003-01-01  TN    temp                         A              Bundled                      temp   
                                                                             Bundled                      temp   
                                                                             Bundled                      temp   
               2005-01-01  TN    temp                         A              Bundled                      temp   
                                                                             Bundled                      temp   
                                                                             Bundled                      temp   
                                                                             Bundled                      temp   
               2006-01-01  TN    temp                         A              Bundled                      temp   
                                                                             Bundled                      temp   
                                                                             Bundled                      temp   
                                                                             Bundled                      temp   
               2007-01-01  TN    temp                         A              Bundle                          0   

                                                                                          commercial_sales_revenue  \
utility_id_eia report_date state balancing_authority_code_eia business_model service_type                            
306.0          2003-01-01  TN    temp                         A              Bundled                          temp   
                                                                             Bundled                          temp   
                                                                             Bundled                          temp   
               2005-01-01  TN    temp                         A              Bundled                          temp   
                                                                             Bundled                          temp   
                                                                      

In [184]:
df_small = raw_df[['utility_id_eia', 'state', 'balancing_authority_name_eia', 'utility_name_eia']].drop_duplicates()

df = (
    df_small
    .groupby(['utility_id_eia'])
    .agg(count=('utility_name_eia', 'count'))
    .reset_index()
)
df2 = df[df['count']>1]
df2

,utility_id_eia,count
0,34,4
1,55,3
2,59,7
3,84,2
4,87,5
...,...,...
3230,57483,2
3236,58119,2
3246,58230,2
3249,88888,3


In [186]:
test = raw_df[raw_df['utility_id_eia']==34].copy()
test

,balancing_authority_code_eia,balancing_authority_id_eia,balancing_authority_name_eia,report_year,state,utility_id_eia,utility_name_eia
0,temp,5416,Duke Energy Corporation,2001,temp,34,temp
4406,temp,5416,Duke Energy Corporation,2002,temp,34,Abbeville City of
6068,temp,5416,Duke Energy Corporation,2003,temp,34,Abbeville City of
9626,temp,5416,Duke Energy Corporation,2004,temp,34,Abbeville City of
12340,temp,5416,Duke Energy Corporation,2005,temp,34,Abbeville City of
15498,temp,5416,Duke Energy Corporation,2006,temp,34,temp
18643,temp,5416,Duke Energy Corporation,2007,temp,34,temp
21819,temp,5416,Duke Energy Corporation,2008,temp,34,temp
24988,temp,5416,Duke Energy Corporation,2009,temp,34,temp
28157,temp,5416,Duke Energy Corporation,2010,temp,34,temp


In [23]:
idx_cols = [
    'utility_id_eia',
    'state',
    'balancing_authority_code_eia',
    'report_year',
]

import pudl.constants as pc

In [24]:
tidy_nm, idx_cols = pudl.transform.eia861._tidy_class_dfs(
    raw_df,
    df_name='Net Metering',
    idx_cols=idx_cols,
    class_list=pc.CUSTOMER_CLASSES,
    class_type='customer_class',
)

In [25]:
tidy_nm

,utility_id_eia,state,balancing_authority_code_eia,report_year,current_flow_type,utility_name_eia,customer_class,chp_cogen_capacity_mw,chp_cogen_customers,chp_cogen_sold_to_utility_mwh,energy_displaced_mwh,other_capacity_mw,other_customers,other_sold_to_utility_mwh,pv_capacity_mw,pv_customers,pv_sold_to_utility_mwh,pv_storage_capacity_mw,pv_storage_customers,pv_virtual_capacity_mw,pv_virtual_customers,total_capacity_mw,total_customers,total_sold_to_utility_mwh,wind_capacity_mw,wind_customers,wind_sold_to_utility_mwh
0,856,TX,UNK,2002.0,NaN,AEP Texas Commercial & Ind,commercial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,NaN,NaN,NaN,NaN
1,856,TX,UNK,2002.0,NaN,AEP Texas Commercial & Ind,industrial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114,NaN,NaN,NaN,NaN
2,856,TX,UNK,2002.0,NaN,AEP Texas Commercial & Ind,residential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4,856,TX,UNK,2002.0,NaN,AEP Texas Commercial & Ind,transportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
5,887,MN,UNK,2002.0,NaN,"Arrowhead Electric Coop, Inc",commercial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55534,27058,WY,WACM,2018.0,AC,"High West Energy, Inc",transportation,NaN,NaN,NaN,NaN,.,.,.,.,.,.,.,.,.,.,0,0,0,.,.,.
55535,99999,WY,PACE,2018.0,NaN,Adjustment 2018,commercial,NaN,NaN,NaN,NaN,.,.,.,-0.102,.,.,.,.,.,.,-0.102,.,.,.,.,.
55536,99999,WY,PACE,2018.0,NaN,Adjustment 2018,industrial,NaN,NaN,NaN,NaN,.,.,.,-0.004,.,.,.,.,.,.,-0.004,.,.,.,.,.
55537,99999,WY,PACE,2018.0,NaN,Adjustment 2018,residential,NaN,NaN,NaN,NaN,.,.,.,-0.176,.,.,.,.,.,.,-0.176,.,.,.,.,.


In [77]:
RECOGNIZED_NERC_REGIONS = [
    'ASCC',
    'ECAR',
    'ERCOT',
    'FRCC',
    'HICC',
    'MAAC',
    'MAIN',
    'MAPP',
    'MRO',
    'NPCC',
    'RFC',
    'SERC',
    'SPP',
    'TRE',
    'WECC',
    'WSCC',  # pre-2002 version of WECC
    'MISO',  # unclear whether technically a regional entity, but lots of entries
    'ECAR_MACC',
    'MAPP_WECC',
    'RFC_SERC',
    'SPP_WECC',
    'MRO_WECC',
    'ERCOT_SPP',
    'SPP_TRE',
    'ERCOT_TRE',
    'MISO_TRE',
    'AK',  # Alaska
    'HI',  # Hawaii
    'VI',  # Virgin Islands
    'GU',  # Guam
    'PR',  # Puerto Rico
    'AS',  # American Samoa
    'UNK',
]

In [93]:
np.setdiff1d(RECOGNIZED_NERC_REGIONS, test_list)

array(['ECAR_MACC'], dtype='<U9')

In [89]:
test_list = sorted(test_df['nerc_region'].unique().tolist())

In [94]:
test_df['nerc_region'].unique()

array(['ECAR', 'MAAC', 'UNK', 'NPCC', 'WSCC', 'SERC', 'MAIN', 'MAPP',
       'ERCOT', 'ASCC', 'SPP', 'PR', 'FRCC', 'GU', 'HI', 'WECC',
       'ECAR_MAAC', 'AK', 'VI', 'AS', 'MAPP_WECC', 'MRO', 'RFC',
       'RFC_SERC', 'SPP_WECC', 'MISO', 'TRE', 'MRO_WECC', 'ERCOT_SPP',
       'HICC', 'SPP_TRE', 'ERCOT_TRE', 'MISO_TRE'], dtype=object)

In [37]:
df = raw_df[idx_cols].copy()

In [38]:
idx_cols = [
    'utility_id_eia',
    'state',
    'report_year',
    'nerc_region'
]

In [39]:
idx_no_nerc = idx_cols.copy()
if 'nerc_region' in idx_no_nerc:
    idx_no_nerc.remove('nerc_region')

# Split raw df into primary keys plus nerc region and other value cols
nerc_df = df[idx_cols].copy()
other_df = df.drop('nerc_region', axis=1).set_index(idx_no_nerc)


In [41]:
nerc_df

,utility_id_eia,state,report_year,nerc_region
0,20139.0,IN,2001,[ECAR]
1,7711.0,OH,2001,[MAAC]
2,21260.0,GA,2001,[UNK]
3,21156.0,NY,2001,[NPCC]
4,21152.0,CA,2001,[WSCC]
...,...,...,...,...
52699,62117.0,CN,2018,[UNK]
52700,62776.0,CN,2018,[UNK]
52701,88888.0,TX,2018,[UNK]
52702,88888.0,TX,2018,[TRE]


# Preliminary Data Wrangling
Once all of the data is loaded and looks like it's in good shape, do any initial wrangling that's specific to this particular analysis. This should mostly make use of the higher level functions which were defined above. If this step takes a while, don't be shy about producing `logging` outputs.

## Fuzzy Match Columns

In [85]:
from fuzzywuzzy import fuzz

def fuzzy_match(df, col1, col2):
    # Define base comparison df 
    eia_base_df = (
        pudl_out.utils_eia860()
        #[['report_date', 'utility_id_eia', 'utility_id_pudl', 'utility_name_eia', 'state']]
        .assign(utility_id_eia=lambda x: x.utility_id_eia.astype('Int64'))
    )
    # Shorten df
    #df = df[['utility_id_eia', 'state', 'utility_name_eia', 'report_date']]
    
    # Merge dfs together 
    df_merge = (
        pd.merge(eia_base_df, df, on=['utility_id_eia', 'state'], how='outer', suffixes=('_base', '_861'))
        .set_index(['utility_id_eia', 'state'])
        .assign(utility_name_eia_base=lambda x: x.utility_name_eia_base.astype('string'),
                utility_name_eia_861=lambda x: x.utility_name_eia_861.astype('string'))
    )
    
    # Only run where both columns for utility name have values
    df_merge_no_null = (
        df_merge.loc[
            (df_merge[col1].notnull()) 
            & (df_merge[col2].notnull())].copy()
    )
    
    # Fuzzy match
    df_merge_no_null['fuzzy'] = (
        df_merge_no_null.apply(lambda x: fuzz.ratio(x.utility_name_eia_base, x.new_parent),axis=1)
    )
#     df_merge_no_null = (
#         df_merge_no_null.assign(fuzzy=lambda x: fuzz.ratio(x[col1], x[col2])))
    
    #df_merge_no_null.loc[:,'fuzzy'] = fuzzy_series
    df_merge_no_null = df_merge_no_null.sort_values('fuzzy')
    
    return df_merge_no_null

# DSM Stuff

In [37]:
dsm = eia861_raw_dfs['demand_side_management_eia861'].copy()
#dsm.columns.tolist()

In [50]:
idx_cols = ['utility_id_eia', 'state', 'ba_code', 'report_year']

dsm_test = dsm[['utility_id_eia', 'state', 'ba_code', 'report_year', 
                'residential_annual_load_management_energy_effects ', 
                'commercial_annual_load_management_energy_effects ', 
                'industrial_annual_load_management_energy_effects ',
                'transportation_annual_load_management_energy_effects ',
                'total_annual_load_management_energy_effects ',
]]

dr_test = raw_dr_eia861[['utility_id_eia', 'state', 'balancing_authority_code_eia', 'report_year',
                         'residential_energy_savings_mwh',
                         'commercial_energy_savings_mwh',
                         'industrial_energy_savings_mwh',
                         'transportation_energy_savings_mwh',
                         'total_energy_savings_mwh'
]].rename(columns={'balancing_authority_code_eia': 'ba_code'})

In [84]:
merge = pd.merge(dsm_test, dr_test, on=idx_cols, how='outer')
merge = pudl.helpers.oob_to_nan(merge, 
                       ['residential_annual_load_management_energy_effects ', 
                        'commercial_annual_load_management_energy_effects ', 
                        'industrial_annual_load_management_energy_effects ',
                        'transportation_annual_load_management_energy_effects ',
                        'residential_energy_savings_mwh',
                        'commercial_energy_savings_mwh',
                        'industrial_energy_savings_mwh',
                        'transportation_energy_savings_mwh',
                        'total_energy_savings_mwh',
                        'total_annual_load_management_energy_effects '])

In [71]:
dr_col = 'residential_energy_savings_mwh'
dsm_col = 'residential_annual_load_management_energy_effects '

In [36]:
merge = merge.loc[merge['report_year'].isin(range(2012,2014))]
merge = merge.loc[merge[dr_col].notna()]
#merge = merge.loc[merge[dsm_col].notna()]


NameError: name 'merge' is not defined

In [34]:
#test.plot.scatter('residential_annual_load_management_energy_effects ', 'residential_energy_savings_mwh')

In [33]:
#raw_dr_eia861.columns.tolist()

# Data Analysis and Visualization
* Now that you've got the required data in a usable form, you can tell the story of your analysis through a mix of visualizations, and further data wrangling steps.
* This narrative should be readable, with figures that have titles, legends, and labeled axes as appropriate so others can understand what you're showing them.
* The code should be concise and make use of the parameters and functions which you've defined above when possible. Functions should contain comprehensible chunks of work that make sense as one step in the story of the analysis.